# IMDB Random Forests Regression
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
final_df = pd.read_csv('../data/totalscore_df.csv')

In [3]:
final_df.columns

Index(['movie_title', 'year', 'actors', 'plot', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Thriller', 'War', 'Western', 'avg_vote', 'votes',
       'weighted_average_vote', 'total_votes', 'mean_vote', 'median_vote',
       'votes_1', 'votes_2', 'votes_3', 'votes_4', 'votes_5', 'votes_6',
       'votes_7', 'votes_8', 'votes_9', 'votes_10', 'us_voters_rating',
       'us_voters_votes', 'plot_sentiment', 'director_score', 'actor_score',
       'actress_score', 'total_score'],
      dtype='object')

In [9]:
final_df.shape

(12821, 52)

## Random Forests Regression

In [16]:
features = ['duration', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 
            'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 
            'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'votes', 'votes_1', 'votes_2', 'votes_3', 'votes_4', 'votes_5', 
            'votes_6', 'votes_7', 'votes_8', 'votes_9', 'votes_10', 'us_voters_rating', 'us_voters_votes', 'plot_sentiment', 
            'director_score', 'actor_score', 'actress_score']

In [17]:
X = final_df[features]
y = final_df['total_score']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = .20)

In [19]:
rf = RandomForestRegressor()

In [20]:
print("Cross Val Score:", cross_val_score(rf, X_train, y_train, cv = 5).mean())

Cross Val Score: 0.959922237940885


In [28]:
model_params = {}
count = 0

In [33]:
rf_params = {
    'max_depth': [9, 10, 11],
    'min_samples_leaf': [4, 5, 6],
    'min_samples_split': [3, 4, 5],
    'n_estimators': [175, 200, 225],
}

gs = GridSearchCV(rf, param_grid=rf_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient='index')

model_df

,max_depth,min_samples_leaf,min_samples_split,n_estimators,best_score,training_score,testing_score
model_1,3,3,3,100,0.882108,0.882990,0.880820
model_2,5,5,3,150,0.918440,0.926830,0.914037
model_3,7,6,3,175,0.938680,0.953694,0.936424
model_4,9,5,4,200,0.949936,0.969696,0.949249
model_5,11,4,4,200,0.955774,0.979447,0.956298


## AdaBoostRegressor

In [20]:
model_params_two = {}
count_two = 0

In [27]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor())

ada_params = {
    'base_estimator__max_depth': [11, 12, 13],
    'learning_rate': [.95, 1.],
    'n_estimators': [70, 75, 80],
}

gs_two = GridSearchCV(ada, param_grid=ada_params, cv=3, n_jobs = 12)

gs_two.fit(X_train, y_train)

count_two += 1

gs_two.best_params_['best_score'] = gs_two.best_score_

gs_two.best_params_['training_score'] = gs_two.score(X_train, y_train)

gs_two.best_params_['testing_score'] = gs_two.score(X_test, y_test)

model_params_two[f'model_{count_two}'] = gs_two.best_params_

model_df_two = pd.DataFrame.from_dict(model_params_two, orient='index')

model_df_two

,base_estimator__max_depth,learning_rate,n_estimators,best_score,training_score,testing_score
model_1,3,0.95,20,0.815696,0.861969,0.772930
model_2,5,0.95,30,0.879960,0.953392,0.850893
model_3,7,0.95,40,0.896370,0.983651,0.878017
model_4,9,1.00,45,0.899599,0.992798,0.884188
model_5,11,0.90,55,0.901346,0.995733,0.885692
model_6,12,1.00,70,0.901715,0.996314,0.887713
model_7,13,0.95,70,0.901044,0.996506,0.886476
